In [ ]:
# fetch initial standings data
import requests
import pandas as pd
from bs4 import BeautifulSoup

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select("table.stats_table")[0]

In [ ]:
# scores and fixtures data
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
team_url = team_urls[0]
data_team = requests.get(team_url)
matches = pd.read_html(data_team.text, match="Scores & Fixtures")

In [ ]:
# shooting stats data
soup_team = BeautifulSoup(data_team.text)
links_team = soup_team.find_all('a')
links_team = [l.get("href") for l in links_team]
links_team = [l for l in links_team if l and 'all_comps/shooting' in l]
data = requests.get(f"https://fbref.com{links_team[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.columns = shooting.columns.droplevel()
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data

In [73]:
"""multiple teams for multiple seasons"""
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

years = list(range(2023, 2021, -1))

# several data frames for match logs for one team for one season
all_matches = []

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select("table.stats_table")[0]

    # scores and fixtures data
    links_standings_tables = standings_table.find_all('a')
    links = [l.get("href") for l in links_standings_tables]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data_team = requests.get(team_url)
        matches = pd.read_html(data_team.text, match="Scores & Fixtures")
        
        soup_team = BeautifulSoup(data_team.text)
        links_shooting_stats_tables = soup_team.find_all('a')
        links_team = [l.get("href") for l in links_shooting_stats_tables]
        links_team = [l for l in links_team if l and 'all_comps/shooting' in l]
        
        data = requests.get(f"https://fbref.com{links_team[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Team"] = team_name
        team_data["Season"] = year
        all_matches.append(team_data)
        time.sleep(1)
    print(len(all_matches))

/var/folders/r_/jvt1rv_x24zfn_hh6c6kkt8h0000gn/T/ipykernel_16186/2115693338.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data_team.text, match="Scores & Fixtures")
/var/folders/r_/jvt1rv_x24zfn_hh6c6kkt8h0000gn/T/ipykernel_16186/2115693338.py:39: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/r_/jvt1rv_x24zfn_hh6c6kkt8h0000gn/T/ipykernel_16186/2115693338.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data_team.text, match="Scores & Fixtures")
/var/folders/r_/jvt1rv_x24zfn_hh6c6kkt8h0000gn/T/i

ValueError: No tables found

In [75]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df

# to csv file
match_df.to_csv("fbref_data.csv", index=False)